# Record

This article is my attempt to explain how the utility [Record](https://www.typescriptlang.org/docs/handbook/utility-types.html#recordkeys-type) work:


Seen from the [source](https://github.com/microsoft/TypeScript/blob/2f34e57ed39ba67b4c4b26bbcb6088cf39aa308a/src/lib/es5.d.ts#L1581), here is the implemented of Record:

In [14]:
{
    type MyRecord<K extends keyof any, T> = {
        [P in K]: T;
    };
}

In above, the operator `keyof` returns a union of all type for the keys of the given operand type ([Ref](https://www.typescriptlang.org/docs/handbook/2/keyof-types.html)). 

In this case, the operand type is `any`, the operation returns `string | number | symbol`. 

In [15]:
import {IsEqual} from 'type-fest'
{
    type KeyOfAny = keyof any
    const expect1:IsEqual<KeyOfAny, string | number | symbol> = true 
}

The type parameter:

```
<K extends keyof any>
```
can therefore be rewritten as:

```
<K extends keyof string | number | symbol >
```
The above expression will constraints `K` to one of `string`, `number` or `symbol`:

In [16]:
import {IsEqual} from 'type-fest'  
{
    type IsKeyOfAny<K> = K extends keyof any ? true : false

    const expect1: IsEqual<IsKeyOfAny<string>, true> = true 
    const expect2: IsEqual<IsKeyOfAny<number>, true> = true 
    const expect3: IsEqual<IsKeyOfAny<symbol>, true> = true 

    const expect4: IsEqual<IsKeyOfAny<boolean>, false> = true 
}

In the context of javascript object, `keyof any` represent the type for key that can be use for the object's property. Namely, `string`, `number` or `symbol`.  

We can describe an object using the [index signatures syntax](https://www.typescriptlang.org/docs/handbook/2/objects.html#index-signatures) for each case where the type when the type for key being `string`, `number` or `symbol`:

In [17]:
// For the case when K is string:
{
    type MyObject<T> = {
        [P: string]: T 
    } 
    const o: MyObject<string> = {}
    o['1'] = 'some string'
}

// For the case when K is number:
{
    type MyObject<T> = {
        [P: number]: T 
    } 
    const o: MyObject<string> = {}
    o[2] = 'some string'
}

// For the case when K is symbol:
{
    type MyObject2<T> = {
        [P: symbol]: T 
    } 
    const o: MyObject2<string> = {}
    o[Symbol(1)] = 'some string'
}


The above is very repetative; we can untilize the constraint syntax `K extends string | number | symbol` to avoid repetition. However, this will cause compiler error:

In [18]:
{
    type MyObject<K extends keyof any, T> = {
        [P: K]: T 
    //   ^--- Error: An index signature parameter type cannot be a literal type or generic type. 
    //               Consider using a mapped object type instead.
    } 

    const o1: MyObject<symbol, string> = {}
    o1['1'] = 'some string'

    const o2: MyObject<symbol, string> = {}
    o2[1] = 'some string'

    const o3: MyObject<symbol, string> = {}
    o2[Symbol(1)] = 'some string'

    console.log(o1, o2, o3)
}


3:10 - An index signature parameter type cannot be a literal type or generic type. Consider using a mapped object type instead.


The typescript compiler does not let you use generic parameter `K` as a type of index `P`. It suggest you to use a mapped object type: 

In [ ]:
{
    type MyObject<K extends keyof any, T> = {
        [P in K]: T 
    } 

    const o1: MyObject<symbol, string> = {}
    o1['1'] = 'some string 1'

    const o2: MyObject<symbol, string> = {}
    o2[1] = 'some string 2'

    const o3: MyObject<symbol, string> = {}
    o3[Symbol(1)] = 'some string 2'

    console.log(o1)
    console.log(o2)
    console.log(o3)
}

Note that for an javascript object, key as type `number` and type `string` are treated as a same key of record:

In [ ]:
{
    const foo = {
        '1': 'string',
        1: 'number'
    //  ^--- Error: Duplicate identifier '1' 
    }
}

toread:

https://stackoverflow.com/questions/65834261/initialize-empty-object-in-typescript-with-record-type

https://stackoverflow.com/questions/70367555/how-to-define-an-empty-object-in-typescript

https://www.google.com/search?q=assign+empty+object+to+Record+site:stackoverflow.com&sxsrf=APwXEddQMWKlTgiIU8RsaneUajlXWL-xzg:1682195166270&sa=X&ved=2ahUKEwjCh9Cqqb7-AhW8kokEHS8JCZYQrQIoBHoECBQQBQ&biw=1453&bih=864&dpr=1